In [23]:
from kekas import Keker, DataOwner, DataKek
from kekas.transformations import Transformer, to_torch, normalize
from kekas.metrics import accuracy, bce_accuracy
from kekas.modules import Flatten, AdaptiveConcatPool2d
from kekas.callbacks import Callback, Callbacks, DebuggerCallback
import pretrainedmodels as pm
from albumentations import Compose, JpegCompression, CLAHE, RandomRotate90, Transpose, ShiftScaleRotate, \
        Blur, OpticalDistortion, GridDistortion, HueSaturationValue, Flip, VerticalFlip

In [28]:
import argparse
from itertools import islice
import json
from pathlib import Path
import shutil
import warnings
from typing import Dict

import numpy as np
import pandas as pd
from sklearn.metrics import fbeta_score
from sklearn.exceptions import UndefinedMetricWarning
import torch
from torch import nn, cuda
from torch.optim import Adam
from torchvision import transforms
import tqdm

from imet.models import get_model
from imet.dataset import TrainDataset, TTADataset, get_ids, DATA_ROOT
from imet.transforms import train_transform, test_transform
from imet.utils import (
    write_event, load_model, mean_df,
    ON_KAGGLE, set_models_path_env, seed_everything, 
    _reduce_loss, _make_mask, binarize_prediction, N_CLASSES)
from imet.losses import loss_function
from imet.optimizers import optimizer
import cv2
from PIL import Image
from torch.utils.data import DataLoader
import os
from imet.losses import loss_function

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
batch_size = 32
input_size = 224
fold = 0
model = 'resnet34'

In [30]:
folds = pd.read_csv('folds.csv')
train_root = os.path.join(DATA_ROOT, 'train')
train_fold = folds[folds['fold'] != fold]
valid_fold = folds[folds['fold'] == fold]

In [31]:
def reader_fn(i, row):
    image = cv2.imread(os.path.join(train_root, f'{row["id"]}.png'))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    labels = torch.zeros(N_CLASSES)
    for cls in row["attribute_ids"].split():
        labels[int(cls)] = 1
    return {"image": image, "labels": labels}


def get_transforms(dataset_key, size, p):
    # we need to use a Transformer class to apply transformations to DataKeks elements
    # dataset_key is an image key in dict returned by reader_fn
    
    TRAIN_AUGS = Transformer(dataset_key, train_transform(input_size))
    VAL_AUGS = Transformer(dataset_key, test_transform(input_size))
                                                      
    TO_ARRAY = Transformer(dataset_key, lambda x: np.array(x))

    NRM_TFMS = transforms.Compose([
        Transformer(dataset_key, to_torch()),
        Transformer(dataset_key, normalize())
    ])
    
    train_tfms = transforms.Compose([TRAIN_AUGS, TO_ARRAY, NRM_TFMS])
    val_tfms = transforms.Compose([VAL_AUGS, TO_ARRAY, NRM_TFMS])  # because we don't want to augment val set yet
    
    return train_tfms, val_tfms

In [32]:
train_tfms, val_tfms = get_transforms("image", input_size, 0.5)

train_dk = DataKek(df=train_fold, reader_fn=reader_fn, transforms=train_tfms)
val_dk = DataKek(df=valid_fold, reader_fn=reader_fn, transforms=val_tfms)

train_dl = DataLoader(train_dk, batch_size=batch_size, num_workers=6, shuffle=True, drop_last=True)
val_dl = DataLoader(val_dk, batch_size=batch_size, num_workers=6, shuffle=False)

In [33]:
dataowner = DataOwner(train_dl, val_dl, None)
criterion = loss_function("fbeta")

In [34]:
def step_fn(model: torch.nn.Module,
            batch: torch.Tensor) -> torch.Tensor:
    """Determine what your model will do with your data.

    Args:
        model: the pytorch module to pass input in
        batch: the batch of data from the DataLoader

    Returns:
        The models forward pass results
    """
    
    # you could define here whatever logic you want
    inp = batch["image"]  # here we get an "image" from our dataset
    return model(inp)

In [35]:
def fbeta(target: torch.Tensor,
          preds: torch.Tensor,
          thresh: bool = 0.1,
          beta: float = 2) -> float:
    target = target.cpu().detach().numpy()
    preds = (torch.sigmoid(preds).cpu().detach().numpy() > thresh).astype(int)
    return fbeta_score(target, preds, beta=beta, average='samples')

In [36]:
model = get_model(model, num_classes=N_CLASSES, pretrained=True, input_size=input_size)

In [37]:
keker = Keker(model=model,
              dataowner=dataowner,
              criterion=criterion,
              step_fn=step_fn,                 
              target_key="labels",               
              metrics={"acc": bce_accuracy, "fbeta": fbeta},      
              opt=torch.optim.SGD).to_fp16()

In [38]:
keker.kek_one_cycle(max_lr=1e-3, cycle_len=1, momentum_range=(0.95, 0.85), div_factor=25)

Epoch 1/1:   9% 256/2731 [00:35<05:23,  7.65it/s, loss=1.6705]


KeyboardInterrupt: 

In [17]:
keker.unfreeze()

In [ ]:
keker.kek_lr(final_lr=3, logdir="logdir/resnet50/3")

In [ ]:
keker.plot_kek_lr("logdir/resnet50/3")

In [ ]:
keker.kek_one_cycle(max_lr=0.5, cycle_len=40, momentum_range=(0.95, 0.85), div_factor=25)

In [ ]:
keker.kek_lr(final_lr=4, logdir="logdir/resnet50/4")
keker.plot_kek_lr("logdir/resnet50/4")

In [ ]:
keker.kek_one_cycle(max_lr=0.5, cycle_len=10, momentum_range=(0.95, 0.85), div_factor=25)